# Lecture 23: High Performance Computing and Cython

### Please note: This lecture will be recorded and made available for viewing online. If you do not wish to be recorded, please adjust your camera settings accordingly. 

# Reminders/Announcements:
- Assignment 8 is due tomorrow at 8pm Pacific (last one!)
- Please do CAPES!
- Final Project "examples" from last year available
    - Warning: I make no claims that these are "perfect" projects (in particular, they are missing participation checks, etc.) but I think the presentations are quite good
- Some final project remarks:
    - If you have tried getting in touch with your groupmates and have had no response, please email me
    - I recommend *making a plan* with your groupmates by the end of next week (things will be collected on Wednesday of finals week, so presentations should occur *before* then)

## Python: Good and Bad

At the very beginning of the class we mentioned some benefits of Python: it is a very flexible language, it has syntax which is easy to read, etc. etc. Some of this comes at a cost though! It is *very* slow. This comes from two main factors:
- The code is *interpreted* instead of *compiled*
- The code is *dynamically typed*. 

Being "interpreted" is the process of translating your typed out syntax into something that a computer can execute (bytecode). This is similar to "compiling" but interpreted code is platform independent, whereas compilation is specific to your local machine. The following StackExchange post: https://stackoverflow.com/questions/1694402/why-are-interpreted-languages-slow has a good analogy:

> If you can talk in your native language to someone, that would generally work faster than having an interpreter having to translate your language into some other language for the listener to understand.

In other words, two native Spanish speakers will communicate to each other more quickly than a Spanish speaker talking to an English speaker using an interpreter.

Being "dynamically typed" means that variables in Python can be *anything* you want them to be (essentially). 

In [1]:
#Integer
x = 5
print(x,type(x))
#List
x = [5]
print(x,type(x))
#String
x = '5'
print(x,type(x))
#Function (This is a stupid example that I just really like, but in general function assignments to a variable *CAN BE* advantageous)
x = print
x(x,type(x))
#Real number
x = 5.0
print(x,type(x))

5 <class 'sage.rings.integer.Integer'>
[5] <class 'list'>
5 <class 'str'>
<built-in function print> <class 'builtin_function_or_method'>
5.00000000000000 <class 'sage.rings.real_mpfr.RealLiteral'>


Dynamic typing allows a lot of flexibility in your code, but that comes at a price. Think about the following code blocks:

In [2]:
x = 5
y = 7
print(x+y)

12


In [3]:
x = [5]
y = [7]
print(x+y)

[5, 7]


In [4]:
x = '5'
y = '7'
print(x+y)

57


The "instructions" of addition here are the *exact same*, but the addition operator *works differently* in each circumstance. This was called *overloading*. Because the machine that executes your code does not know what type your variables are, to execute `x+y` it has to:
- first look up the type of x
- then look up the type of y
- ensure that these can be "added" in a consistent way
- potentially convert x or y to allow for this
- figure out how to add them
- add them

Does this really have a huge impact on runtime? Well, let's compare Python to...

## Cython

From Wikipedia: https://en.wikipedia.org/wiki/Cython, Cython is "a programming language that aims to be a superset of the Python programming language, designed to give C-like performance with code that is written mostly in Python with optional additional C-inspired syntax."

It is a *compiled* language which *looks* similar to Python but *performs* more closely to C. 

In the Sage kernel, we will actually be able to compare Python and Cython *directly* in the same file.

Let's add the numbers from 1 to 1000000 in both languages. First in Python:

In [5]:
import time
t = time.time() # Current time in seconds
print('hello world!')
m = sum(1 for _ in range(1000000))
print(time.time()-t) # Elapsed time in seconds

hello world!
0.14260411262512207


Now we will load Cython (made possible by working in Sage)

In [6]:
%load_ext Cython

And we will do the same calculation in Cython using %%cython

In [7]:
%%cython

import time
t = time.time()
print('hello world')
m = sum(1 for _ in range(1000000))
print(time.time()-t)

hello world
0.01252889633178711


Note that there was a considerable *lag* in the execution of the above cell; this is due to the compilation. But this is a one time cost. In general once you compile something you can run it again directly as much as you want. 

The key takeaway is that the internal timing itself was *much faster* (depending on the specific run, I obtained speedups of ~2x - 10x). And this is only the very very beginning. 

## A Word of Warning

In your day to day programming life, you *may not need* the speedups that languages like Cython or C offer. In particular, a *very important* thing to take into account is the *human factor* of programming. If optimizing your code takes 10 hours of frustration, and you only shave off 10 seconds on the runtime, that may not be worth it.

## *********** Participation Check ********************
Run the command `xkcd(1205)` in the code cell below. If you do a task once a week for 5 years and can shave off 30 seconds on doing that task, how much time is worth spending on optimizing that task?

In [8]:
xkcd(1205)

<h1>Is It Worth the Time?</h1><img src="https://imgs.xkcd.com/comics/is_it_worth_the_time.png" title="Don't forget the time you spend finding the chart to look up what you save. And the time spent reading this reminder about the time spent. And the time trying to figure out if either of those actually make sense. Remember, every second counts toward your life total, including these right now."><div>Source: <a href="http://xkcd.com/1205" target="_blank">http://xkcd.com/1205</a></div>

Answer here: 
## ********************************************************

A quote attributed to Donald Knuth: "Premature optimization is the root of all evil."

Python *isn't slow/bad at everything*.
- Iterators are great at saving on memory
- Some Python functions essentially call underlying C code directly (for instance, the `pow` function) and are already quite fast
- Numpy arrays are designed to be efficient already
- Certain parts of libraries (such as Sage) already use Cython for speedups

In the remainder of this lecture we will try to show *when* such optimizations can be fruitful, and how to achieve that. 

## Static Typing

The main/easiest speedup that Cython offers is static typing. Let's start with the following, which is "just Python" running in the Cython environment.

In [9]:
%%cython
import time
t = time.time()
a = 0
for i in range(10000):
    a += i
print(a)
print(time.time()-t)

49995000
0.0010123252868652344


Note that the variable `a` is dynamically typed. Let's add a short modification:

In [10]:
%%cython
import time
t = time.time()
cdef int a = 0
for i in range(10000):
    a += i
print(a)
print(time.time()-t)

49995000
0.0009799003601074219


The extra `cdef int` tells Cython "I only want `a` to be an integer." This allows Cython to speedup interactions with `a` at the cost of some flexibility:

In [11]:
%%cython
import time
t = time.time()
cdef int a = 0
a = 'string'
print(a)
print(time.time()-t)


Error compiling Cython file:
------------------------------------------------------------
...
import time
t = time.time()
cdef int a = 0
a = 'string'
   ^
------------------------------------------------------------

/home/user/.cache/ipython/cython/_cython_magic_c0bae56b5871c8948ffcbf3a71a2018b.pyx:4:4: Unicode literals do not support coercion to C types other than Py_UNICODE/Py_UCS4 (for characters) or Py_UNICODE* (for strings).


In general, if you want a "quick" way to try and get faster runtime, you can try to statically type your variables.

## Annotating

Ok, what if you've tried a quick pass at making your code more efficient, and want other ideas? Cython allows an `annotate` tag which can help:

In [12]:
%%cython --annotate
import time
t = time.time()
cdef int a = 0
for i in range(10000):
    a += i
print(a)
print(time.time()-t)

49995000
0.0015342235565185547


A general rule is that "more yellow" is "more slow." It's not always possible to do something about this, but in this short example there *is* a big thing we can do:

In [13]:
%%cython --annotate
import time
t = time.time()
cdef int a = 0
cdef int i
for i in range(10000):
    a += i
print(a)
print(time.time()-t)

49995000
9.107589721679688e-05


Note that we didn't have to assign the variable `i` to anything; all we had to do is tell Cython that `i` would be an integer.

Another warning: We're not only telling Cython that `i` is an integer; we're declaring it to be a *C integer*. In general, using `cdef` creates *static C typed* variables. A result of this is that they are subject to C restrictions. For instance; an integer in Python cannot "overflow" (get too big) but a C integer can:

In [14]:
%%cython

cdef int a = 1
for _ in range(24):
    a *= 4
    print(a)

4
16
64
256
1024
4096
16384
65536
262144
1048576
4194304
16777216
67108864
268435456
1073741824
0
0
0
0
0
0
0
0
0


## Function Calls 

The use of `cdef` can extend to function calls. The following code approximates the integral of `f(x) = x^2 - x` on an interval `[a,b]`

In [15]:
# Sage version (with Rationals)

def f(x):
    return x ** 2 - x

def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

import time
t = time.time()
m = integrate_f(0, 1, 1000000)
print(m)
print(time.time()-t)

-333333333333/2000000000000
2.722400665283203


In [16]:
# Python version

def f(x):
    return x ** 2 - x

def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

import time
t = time.time()
m = integrate_f(int(0), int(1), int(1000000))
print(m)
print(time.time()-t) #Good improvement...

-0.1666666666665057
0.37911486625671387


In [17]:
%%cython

#Python version, in cython
def f(x):
    return x ** 2 - x

def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

import time
t = time.time()
m = integrate_f(0, 1, 1000000)
print(m)
print(time.time()-t) # Some more improvement...

-0.1666666666665057
0.33864426612854004


## ***** Participation Check *****************
Look at the code cell above. What variables are we using that could have a `cdef` applied to them?

List them here: 

## ****************************************

In [18]:
%%cython 

#Cython with basic static typing
def f(double x):
    return x ** 2 - x

def integrate_f(double a,double b,int N):
    cdef int i
    cdef double s
    cdef double dx
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

import time
t = time.time()
m = integrate_f(0, 1, 1000000)
print(m)
print(time.time()-t) # Some more improvement...

-0.1666666666665057
0.04636359214782715


In [19]:
%%cython

#Cython with static typing on functions as well
cdef double f(double x):
    return x ** 2 - x

cdef double integrate_f(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

import time
t = time.time()
m = integrate_f(0, 1, 1000000)
print(m)
print(time.time()-t) # Way better!

-0.1666666666665057
0.004075288772583008


In testing during prep for the lecture, I got ~1500x speedup from Sage and ~100x speedup from Python!

In [20]:
%%cython --annotate

def f(x):
    return x ** 2 - x

def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

import time
t = time.time()
m = integrate_f(0, 1, 1000000)
print(m)
print(time.time()-t) 

-0.1666666666665057
0.1724872589111328


In [21]:
%%cython --annotate

cdef double f(double x):
    return x ** 2 - x

cdef double integrate_f(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

import time
t = time.time()
m = integrate_f(0, 1, 1000000)
print(m)
print(time.time()-t) 

-0.1666666666665057
0.004286289215087891


## A Catch

A problem with `cdef` is that it *only* defines it with respect to Cython. You will not be able to call it "in Python"

In [22]:
%%cython

cdef double f(double x):
    return x ** 2 - x

cdef c_integrate_f(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

print(c_integrate_f(0,1,1000))

-0.1666664999999999


In [23]:
#This should fail:
print(c_integrate_f(0,1,1000))

NameError: name 'c_integrate_f' is not defined

One option would be to write separate instances for Cython and Python, but that's just messy. Instead you can use `cpdef`; this creates two versions of the function. A "fast C version" and a "slow Python version"

In [24]:
%%cython

cdef double f(double x):
    return x ** 2 - x

cpdef cp_integrate_f(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

In [25]:
print(cp_integrate_f(0,1,1000))

-0.1666664999999999


## Bottlenecks

A key use of optimization is to reduce "bottlenecks"

![](tr.png)

For this road, you shouldn't bother trying to make the highway paved better on either end; you should focus on adding more lanes into the middle section.

Trying to find the bottlenecks in your code is called "profiling." Cython has a tutorial here: https://cython.readthedocs.io/en/latest/src/tutorial/profiling_tutorial.html

There are many useful tools for this. An old timey way would be simply to add print statements in the middle of your functions to allow for granular timing; this does not scale well though and is a fairly crude technique. Jupyter provides a fairly nice one which is much nicer: the `prun` command. We will use it to approximate $\pi$ via the following formula:

$$
\sum_{k=1}^\infty\frac{1}{k^2} = \frac{\pi^2}{6}.
$$


In [26]:
def recip_square(i):
    return 1. / i ** 2

def approx_pi(n=10000000):
    val = 0.
    for k in range(1, n + 1):
        val += recip_square(k)
    return (6 * val) ** .5

In [27]:
%time approx_pi()

CPU times: user 25.8 s, sys: 28.7 ms, total: 25.8 s
Wall time: 26.7 s


3.14159255809590

In [28]:
prun approx_pi()

         20000006 function calls in 31.827 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
 10000000   14.779    0.000   26.247    0.000 <ipython-input-26-830e9e77724b>:1(recip_square)
 10000002   11.468    0.000   11.468    0.000 {sage.rings.real_mpfr.create_RealNumber}
        1    5.580    5.580   31.827   31.827 <ipython-input-26-830e9e77724b>:4(approx_pi)
        1    0.000    0.000   31.827   31.827 {built-in method builtins.exec}
        1    0.000    0.000   31.827   31.827 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

We can see that a lot of time is spent computing reciprocal squares and "creating real numbers"! Let's start by statically typing some things:

In [29]:
%%cython

# cython: profile=True

def c_recip_square(int i):
    return 1. / i ** 2

def c_approx_pi(int n=10000000):
    cdef double val = 0.
    cdef int k
    for k in range(1, n + 1):
        val += c_recip_square(k)
    return (6 * val) ** .5

In [30]:
prun c_approx_pi()

         10000005 function calls in 2.115 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.528    1.528    2.107    2.107 _cython_magic_0f6ccefb12b9989158422409b7373dae.pyx:7(c_approx_pi)
 10000000    0.579    0.000    0.579    0.000 _cython_magic_0f6ccefb12b9989158422409b7373dae.pyx:4(c_recip_square)
        1    0.008    0.008    2.115    2.115 <string>:1(<module>)
        1    0.000    0.000    2.107    2.107 {_cython_magic_0f6ccefb12b9989158422409b7373dae.c_approx_pi}
        1    0.000    0.000    2.115    2.115 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

Can we do better?

In [31]:
%%cython

# cython: profile=True

cdef double c2_recip_square(int i):
    
    return 1. / (i*i)

def c2_approx_pi(int n=10000000):
    cdef double val = 0.
    cdef int k
    for k in range(1, n+1):
        val += c2_recip_square(k)
    return (6 * val) ** .5

In [32]:
prun c2_approx_pi()

ZeroDivisionError: float division

Exception ignored in: '_cython_magic_87394d902e1f769461d6cd097f7df3ba.c2_recip_square'
Traceback (most recent call last):
  File "<string>", line 1, in <module>
ZeroDivisionError: float division


ZeroDivisionError: float division

Exception ignored in: '_cython_magic_87394d902e1f769461d6cd097f7df3ba.c2_recip_square'
Traceback (most recent call last):
  File "<string>", line 1, in <module>
ZeroDivisionError: float division


ZeroDivisionError: float division

...

ZeroDivisionError: float division

Exception ignored in: '_cython_magic_87394d902e1f769461d6cd097f7df3ba.c2_recip_square'
Traceback (most recent call last):
  File "<string>", line 1, in <module>
ZeroDivisionError: float division


ZeroDivisionError: float division

Exception ignored in: '_cython_magic_87394d902e1f769461d6cd097f7df3ba.c2_recip_square'
Traceback (most recent call last):
  File "<string>", line 1, in <module>
ZeroDivisionError: float division


         10099804 function calls (10098892 primitive calls) in 7.285 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    3.133    3.133    7.285    7.285 _cython_magic_87394d902e1f769461d6cd097f7df3ba.pyx:8(c2_approx_pi)
 10000000    1.752    0.000    4.152    0.000 _cython_magic_87394d902e1f769461d6cd097f7df3ba.pyx:4(c2_recip_square)
     3935    0.926    0.000    0.926    0.000 {method 'acquire' of '_thread.lock' objects}
      152    0.588    0.004    1.970    0.013 interactiveshell.py:1935(excepthook)
     2595    0.527    0.000    0.527    0.000 socket.py:357(send)
      608    0.070    0.000    0.095    0.000 encoder.py:304(iterencode)
      152    0.060    0.000    0.060    0.000 inspect.py:1496(getinnerframes)
      152    0.053    0.000    0.053    0.000 traceback.py:321(extract)
     4864    0.011    0.000    0.012    0.000 {method 'sub' of 're.Pattern' objects}
     2595    0.011    0.000    0.550    0.0

Be careful!!!

In [0]:
c2_approx_pi()

In [33]:
%%cython

# cython: profile=True

cdef double c2_recip_square(int i):
    cdef double j = i
    return 1. / (j*j)

def c2_approx_pi(int n=10000000):
    cdef double val = 0.
    cdef int k
    for k in range(1, n+1):
        val += c2_recip_square(k)
    return (6 * val) ** .5

In [34]:
c2_approx_pi()

3.1415925580959025

In [35]:
prun c2_approx_pi()

         10000005 function calls in 1.833 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.198    1.198    1.833    1.833 _cython_magic_05c08c92d3752c7b2ad7a2acc9f6b4ff.pyx:8(c2_approx_pi)
 10000000    0.635    0.000    0.635    0.000 _cython_magic_05c08c92d3752c7b2ad7a2acc9f6b4ff.pyx:4(c2_recip_square)
        1    0.000    0.000    1.833    1.833 <string>:1(<module>)
        1    0.000    0.000    1.833    1.833 {built-in method builtins.exec}
        1    0.000    0.000    1.833    1.833 {_cython_magic_05c08c92d3752c7b2ad7a2acc9f6b4ff.c2_approx_pi}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

This improvement looks good, but not *that good*. That's because a lot of overhead is introduced from the profiler. If we just time it:

In [36]:
%time c2_approx_pi() 

CPU times: user 39.4 ms, sys: 0 ns, total: 39.4 ms
Wall time: 39.1 ms


3.1415925580959025

We get a *much* better result!

## One More Profiling Technique: lprun

You can also profile *line by line* if you are really serious about this (again; not always necessary for you to get this serious). Let's go back to sets vs lists!

In [0]:
def sum1(S,bad):
    j = 0
    for i in S:
        if i not in bad:
            j+=i

In [0]:
%load_ext line_profiler

In [0]:
lprun -f sum1 sum1({i for i in range(100000)}, {i**2 for i in range(10000)})

In [0]:
lprun -f sum1 sum1([i for i in range(100000)], [i**2 for i in range(10000)])

What we see from the above examples is that lookup in sets is *much much faster* than lookup in lists. So maybe you want to use this knowledge. Be careful! If you do this carelessly, you'll do worse!

In [0]:
def sum2(S,bad):
    j = 0
    for i in S:
        if i not in set(bad):
            j+=i

In [0]:
lprun -f sum2 sum2([i for i in range(100000)], [i**2 for i in range(10000)])

What's going on here!?!?!

In [0]:
def sum3(S,bad):
    j = 0
    T = set(bad)
    for i in S:
        if i not in T:
            j+=i

In [0]:
lprun -f sum3 sum3([i for i in range(100000)], [i**2 for i in range(10000)])

Muuuuch better

## What's Next?

Profiling is a subtle art, and speeding up your code can be achieved in *many* different ways which we won't get into. 

A common one that you may want to look into is *parallelization*. Suppose I chose 5 of you and gave your group 5 lists of numbers. Your task is to give me the sum of all the numbers in those 5 lists. You could either:
- pick an unlucky member of your group to add them all, while four of you gossip about flowers or something
- each take one of the lists and add the numbers in that list. Then get the final result by adding your individual results.

There is a clear winner here (Option A, I hate adding and would prefer one of you to do it for me)

This is the idea of "parallelization." Split your job into multiple "threads" and have a worker for each thread. Then combine your results from each worker in the end. 

True parallelization is not directly allowed by Python, as it uses a *Global Interpreter Lock*:https://en.wikipedia.org/wiki/Global_interpreter_lock. There are various reasons for this; chief among them are that often processes are not "thread safe" unless you are careful. 

You *can* get around this using Cython if you know what you are doing. Python code which utilizes Cython can allow parallel execution using OpenMP. I don't know nearly enough about this to teach you anything useful, but here are the docs if you are interested: https://cython.readthedocs.io/en/latest/src/userguide/parallelism.html

## Julia

Another thing you may want to explore is the Julia programming language: https://julialang.org/ . It is supposed to be a "Python ish" language which comes with many of these speedups builtin, but I know *nothing about it*. Word on the street is that Math157 *may* transition to Julia one day, but who knows.